In [26]:
!uv pip install openai==1.3.5
!uv pip install python-dotenv

Resolved 16 packages in 637ms                                        
⠙ Preparing packages... (0/2)                                                   
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
anyio                ------------------------------     0 B/79.00 KiB
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
anyio                ------------------------------ 14.93 KiB/79.00 KiB
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
anyio                ------------------------------ 30.93 KiB/79.00 KiB
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
anyio                ------------------------------ 46.93 KiB/79.00 KiB
⠙ Preparing packages... (0/2)--------------     0 B/215.62 KiB          
anyio                ------------------------------ 46.93 KiB/79.00 KiB
⠙ Preparing packages... (0/2)-------------- 2.72 KiB/

In [28]:
import os
import re
import json
import time
import openai
import dotenv
from dotenv import load_dotenv
from openai import OpenAI

In [29]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [30]:
# 2. Hàm gọi GPT để gán label cho một record
def extract_json_list(text):
    """
    Tìm và trả về JSON list đầu tiên trong chuỗi (ngay cả khi có bọc ```json)
    """
    # Nếu được bọc trong ```json ``` thì loại bỏ
    match = re.search(r"```(?:json)?\s*(\[.*?\])\s*```", text, re.DOTALL)
    if match:
        return match.group(1)

    # Hoặc tìm JSON list thuần trong văn bản
    match = re.search(r"(\[.*?\])", text, re.DOTALL)
    if match:
        return match.group(1)

    raise ValueError("Không tìm thấy JSON list trong content GPT trả về.")

def label_options(record, model="gpt-4o", temperature=0.0):
    prompt = f"""
Bạn là trợ lý AI, nhiệm vụ phân loại các phương án trả lời về thời lượng:
- Input là JSON có fields "context", "question", "options" (là list các string)
- Output chỉ trả về JSON list "labels" có cùng độ dài với "options", giá trị "yes" nếu option đúng mô tả thời lượng, ngược lại "no".

Ví dụ:
Input:
{{
  "context": "Tôi đang sửa chữa chiếc xe đạp bị hỏng.",
  "question": "Mất bao lâu để sửa chữa chiếc xe đạp?",
  "options": ["30 phút", "1 tháng", "10 phút", "2 giờ"]
}}
Output:
["yes", "no", "yes", "yes"]

Bây giờ phân loại cho record sau:
{json.dumps(record, ensure_ascii=False)}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Bạn là trợ lý AI chuyên gán label yes/no."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=100,
    )

    content = response.choices[0].message.content.strip()

    try:
        # Trích xuất phần JSON list hợp lệ
        json_text = extract_json_list(content)
        labels = json.loads(json_text)
        assert isinstance(labels, list)
        return labels
    except Exception as e:
        print(f"[LỖI PARSE JSON] Content không hợp lệ:\n{content}")
        raise e


In [ ]:
def main(input_path="public_test.txt", output_path="public_test_gpt_4o.txt"):
    with open(input_path, "r", encoding="utf-8") as fin, \
         open(output_path, "w", encoding="utf-8") as fout:
        for line in fin:
            line = line.strip()
            if not line:
                continue
            record = json.loads(line)
            # Gọi GPT gán labels
            try:
                labels = label_options(record)
            except Exception as e:
                print(f"Error when labeling QID={record.get('qid')}: {e}")
                # nếu lỗi, có thể thử chờ rồi gọi lại
                time.sleep(5)
                labels = label_options(record)
            # Gắn thêm trường labels
            record["labels"] = labels
            # Ghi ra file (mỗi dòng 1 JSON)
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")
            # tránh call quá nhanh
            time.sleep(0.5)

In [ ]:
if __name__ == "__main__":
    main()
    print("Hoàn thành!")

KeyboardInterrupt: 